In [65]:
import pandas as pd
import numpy as np

import pickle 
import os
import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tqdm import tqdm

import tokenize

In [2]:
with open(os.path.join('../data', 'repos_as_pandas_metainfo_20230126.pkl'), 'rb') as file:
  repos = pickle.load(file)

In [9]:
repos = repos[repos.type_script=='py']

In [21]:
def standardize_types(x):
    if isinstance(x, list):
        return ','.join(x)
    else:
        return x

def f(x):
    if isinstance(x, list):
        return ','.join(x)
    else:
        return x

In [23]:
repos.meta_keywords = repos.meta_keywords.apply(standardize_types)

/Users/izzy/anaconda3/envs/encode_code/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
vectorizer = TfidfVectorizer(max_features=100, stop_words='english', use_idf=True)

In [28]:
keywords_fequencies = pd.DataFrame(mat.sum(axis=0).T,
             index=vectorizer.get_feature_names(),
             columns=['freq']).sort_values(by='freq',
                                           ascending=False)

/Users/izzy/anaconda3/envs/encode_code/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
keywords_fequencies

,freq
analysis,83.284107
data,64.875023
plot,57.757637
cryptocurrency,53.324372
sentiment,52.518293
...,...
pages,8.733238
local,8.660670
greeks,7.686056
delta,7.423465


In [32]:
tokens = tokenize.generate_tokens(repos.code_script.iloc[0])

In [33]:
tokens = tokenize.generate_tokens(repos.code_script.iloc[0])

<generator object _tokenize at 0x7f8891a34eb0>

In [61]:
def tokenize_code(df, idx, name_only=True):
    tokens_list = []
    type_list = []
    with tokenize.open(os.path.join(df['folder_name'].iloc[idx], df['script_name'].iloc[idx])) as f:
        tokens = tokenize.generate_tokens(f.readline)
        for token in tokens:
            if name_only:
                if token.type==1:
                    tokens_list.append(token.string) 
            else:
                tokens_list.append(token.string) 
                type_list.append(token.type) 
       
        if name_only:
            return tokens_list
        return (tokens_list, type_list)


In [71]:
repos['qid'] = range(len(repos.index))

/var/folders/9b/jflbvjp576j7v4px00lmp6000000gn/T/ipykernel_99246/2814995925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repos['qid'] = range(len(repos.index))


In [88]:
repos['code_words'] = ''
for idx in tqdm(range(len(repos.index))):
    l = repos.qid.iloc[idx]
    if (l==521) | (l==1252) | (l==1270):
        continue
    repos['code_words'].iloc[idx] = ' '.join(tokenize_code(repos, idx))

/var/folders/9b/jflbvjp576j7v4px00lmp6000000gn/T/ipykernel_99246/1144518057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repos['code_words'] = ''
100%|██████████| 1912/1912 [00:07<00:00, 249.26it/s]


In [91]:
# split snake case 
repos['code_words'] = repos['code_words'].apply(lambda x: x.replace('_', ' '))

/var/folders/9b/jflbvjp576j7v4px00lmp6000000gn/T/ipykernel_99246/2240814312.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repos['code_words'] = repos['code_words'].apply(lambda x: x.replace('_', ' '))


In [99]:
'''# split camel case 
def camel_case_split(str):
    words = [[str[0]]]
 
    for c in str[1:]:
        if words[-1][-1].islower() and c.isupper():
            words.append(list(c))
        else:
            words[-1].append(c)
 
    return [''.join(word) for word in words]

def camel_line(line): 
    new_line = []
    for word in line.split(' '):
        print(word)
        new_line.extend(camel_case_split(word))

repos['code_words'] = repos['code_words'].apply(camel_line)'''

"# split camel case \ndef camel_case_split(str):\n    words = [[str[0]]]\n \n    for c in str[1:]:\n        if words[-1][-1].islower() and c.isupper():\n            words.append(list(c))\n        else:\n            words[-1].append(c)\n \n    return [''.join(word) for word in words]\n\ndef camel_line(line): \n    new_line = []\n    for word in line.split(' '):\n        print(word)\n        new_line.extend(camel_case_split(word))\n\nrepos['code_words'] = repos['code_words'].apply(camel_line)"

In [101]:
vectorizer = TfidfVectorizer(max_features=100, stop_words='english', use_idf=True)
vectorizer.fit(repos.meta_keywords)

TfidfVectorizer(max_features=100, stop_words='english')

In [102]:
keywords_fequencies = pd.DataFrame(mat.sum(axis=0).T,
             index=vectorizer.get_feature_names(),
             columns=['freq']).sort_values(by='freq',
                                           ascending=False)

/Users/izzy/anaconda3/envs/encode_code/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [103]:
vectorizer_code = TfidfVectorizer(max_features=100, stop_words='english', use_idf=True)
vectorizer_code.fit(repos.code_words)

TfidfVectorizer(max_features=100, stop_words='english')

In [104]:
code_fequencies = pd.DataFrame(mat.sum(axis=0).T,
             index=vectorizer_code.get_feature_names(),
             columns=['freq']).sort_values(by='freq',
                                           ascending=False)

/Users/izzy/anaconda3/envs/encode_code/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [105]:
code_fequencies

,freq
append,83.284107
drop,64.875023
pyplot,57.757637
df,53.324372
size,52.518293
...,...
probs,8.733238
min,8.660670
list,7.686056
features,7.423465
